# Part 3: Create a model to serve the item embedding data

This notebook is the third of five notebooks that guide you through running the [Real-time Item-to-item Recommendation with BigQuery ML Matrix Factorization and ScaNN](https://github.com/GoogleCloudPlatform/analytics-componentized-patterns/tree/master/retail/recommendation-system/bqml-scann) solution.

Use this notebook to wrap the item embeddings data in a Keras model that can act as an item-embedding lookup, then export the model as a SavedModel.

Before starting this notebook, you must run the [02_export_bqml_mf_embeddings](02_export_bqml_mf_embeddings.ipynb) notebook to process the item embeddings data and export it to Cloud Storage.

After completing this notebook, run the [04_build_embeddings_scann](04_build_embeddings_scann.ipynb) notebook to create an approximate nearest neighbor index for the item embeddings.



## Setup

Import the required libraries, configure the environment variables, and authenticate your GCP account.

In [ ]:
!pip install -q -U pip
!pip install -q tensorflow==2.2.0
!pip install -q -U google-auth google-api-python-client google-api-core

### Import libraries

In [ ]:
import os
import tensorflow as tf
import numpy as np
print(f'Tensorflow version: {tf.__version__}')

### Configure GCP environment settings

Update the following variables to reflect the values for your GCP environment:

+ `PROJECT_ID`: The ID of the Google Cloud project you are using to implement this solution.
+ `BUCKET`: The name of the Cloud Storage bucket you created to use with this solution. The `BUCKET` value should be just the bucket name, so `myBucket` rather than `gs://myBucket`.

In [ ]:
PROJECT_ID = 'yourProject' # Change to your project.
BUCKET = 'yourBucketName' # Change to the bucket you created.
EMBEDDING_FILES_PATH = f'gs://{BUCKET}/bqml/item_embeddings/embeddings-*'
MODEL_OUTPUT_DIR = f'gs://{BUCKET}/bqml/embedding_lookup_model'

!gcloud config set project $PROJECT_ID

### Authenticate your GCP account
This is required if you run the notebook in Colab. If you use an AI Platform notebook, you should already be authenticated.

In [ ]:
try:
  from google.colab import auth
  auth.authenticate_user()
  print("Colab user is authenticated.")
except: pass

## Create the embedding lookup model

You use the `EmbeddingLookup` class to create the item embedding lookup model. The `EmbeddingLookup` class inherits from [tf.keras.Model](https://www.tensorflow.org/api_docs/python/tf/keras/Model), and is implemented in the
[lookup_creator.py](embeddings_lookup/lookup_creator.py)
module.

The `EmbeddingLookup `class works as follows:

1. Accepts the `embedding_files_prefix` variable in the class constructor. This variable points to the Cloud Storage location of the CSV files containing the item embedding data.     
1. Reads and parses the item embedding CSV files.
1. Populates the `vocabulary` and `embeddings` class variables. `vocabulary` is an array of item IDs, while `embeddings` is a Numpy array with the shape (*number of embeddings*, *embedding dimensions*). 
1. Appends the `oov_embedding` variable to the `embeddings` variable. The `oov_embedding` variable value is all zeros, and it represents the out of vocabulary (OOV) embedding vector. The `oov_embedding` variable is used when an invalid ("out of vocabulary", or OOV) item ID is submitted, in which case an embedding vector of zeros is returned.
1. Writes the `vocabulary` value to a file, one array element per line, so it can be used as a model asset by the SavedModel.
1. Uses `token_to_idx`, a `tf.lookup.StaticHashTable` object, to map the
   item ID to the index of the embedding vector in the `embeddings` Numpy array.
1. Accepts a list of strings with the `__call__` method of the model. Each string represents the item ID(s) for which the embeddings are to be retrieved. If the input list contains _N_ strings, then _N_ embedding vectors are returned. 

    Note that each string in the input list may contain one or more space-separated item IDs. If multiple item IDs are present, the embedding   vectors of these item IDs are retrieved and _combined_ (by averaging)   into a single embedding vector. This makes it possible to fetch an embedding vector representing a set of items (like a playlist) rather than just a single item.

### Clear the model export directory

In [ ]:
if tf.io.gfile.exists(MODEL_OUTPUT_DIR):
  print("Removing {} contents...".format(MODEL_OUTPUT_DIR))
  tf.io.gfile.rmtree(MODEL_OUTPUT_DIR)

### Create the model and export the SavedModel file

Call the `export_saved_model` method, which uses the `EmbeddingLookup` class to create the model and then exports the resulting SavedModel file:

In [ ]:
from embeddings_lookup import lookup_creator
lookup_creator.export_saved_model(EMBEDDING_FILES_PATH, MODEL_OUTPUT_DIR)

Inspect the exported SavedModel using the `saved_model_cli` command line tool:


In [ ]:
!saved_model_cli show --dir {MODEL_OUTPUT_DIR} --tag_set serve --signature_def serving_default

### Test the SavedModel file

Test the SavedModel by loading it and then calling it with input item IDs:


In [ ]:
loaded_model = tf.saved_model.load(MODEL_OUTPUT_DIR)

In [ ]:
input_items = ['2114406', '2114402 2120788', 'abc123']
output = loaded_model(input_items)
print(f'Embeddings retrieved: {output.shape}')
for idx, embedding in enumerate(output):
  print(f'{input_items[idx]}: {embedding[:5]}')

The output shows the output embedding vector (the first five elements of each vector) for each input item. Note the following:

+ The second entry in the input list contains two item IDs, `2114402` and `2120788`. The returned vector is the average of the embeddings of these two items.
+ The third entry in the input list, `abc123`, is an invalid item ID, so the returned embedding vector contains zeros.


## License

Copyright 2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at: http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. 

See the License for the specific language governing permissions and limitations under the License.

**This is not an official Google product but sample code provided for an educational purpose**